In [1]:
! pip install pygame


In [2]:
import pygame
pygame.init()
print("pygame working")
pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
pygame working


In [2]:
import pygame
import random

# ================= CONFIG =================
GRID_SIZE = 6
CELL_SIZE = 80
TOP_BAR = 40
BOTTOM_BAR = 40

WIDTH = GRID_SIZE * CELL_SIZE
HEIGHT = GRID_SIZE * CELL_SIZE + TOP_BAR + BOTTOM_BAR
FPS = 60

# ================= COLORS =================
BG = (18, 18, 22)
BOARD_BG = (30, 30, 40)

PLAYER = (70, 140, 255)
AI = (255, 90, 90)
NEUTRAL = (90, 90, 110)

TEXT = (230, 230, 230)
PLAYER_TURN = (90, 160, 255)
AI_TURN = (255, 120, 120)

# ================= INIT =================
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Strategy AI Game")
clock = pygame.time.Clock()

font = pygame.font.SysFont("arial", 20, bold=True)
big_font = pygame.font.SysFont("arial", 42, bold=True)

# ================= GAME STATE =================
grid = [[NEUTRAL for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
grid[0][0] = PLAYER
grid[GRID_SIZE - 1][GRID_SIZE - 1] = AI

player_turn = True
game_over = False
winner = None

# ================= HELPERS =================
def neighbors(x, y):
    for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE:
            yield nx, ny

def can_capture(x, y, owner):
    if grid[y][x] == owner:
        return False
    return any(grid[ny][nx] == owner for nx, ny in neighbors(x, y))

def count_cells(owner):
    return sum(row.count(owner) for row in grid)

def check_win():
    total = GRID_SIZE * GRID_SIZE
    if count_cells(PLAYER) > total // 2:
        return "PLAYER"
    if count_cells(AI) > total // 2:
        return "AI"
    return None

# ================= DRAW FUNCTIONS =================
def draw_turn_bar():
    bar = pygame.Rect(0, 0, WIDTH, TOP_BAR)
    color = PLAYER_TURN if player_turn else AI_TURN
    pygame.draw.rect(screen, color, bar)

    text = "YOUR TURN" if player_turn else "AI THINKING..."
    label = font.render(text, True, (20, 20, 20))
    screen.blit(label, (WIDTH//2 - label.get_width()//2, 10))

def draw_grid():
    mx, my = pygame.mouse.get_pos()

    for y in range(GRID_SIZE):
        for x in range(GRID_SIZE):
            rect = pygame.Rect(
                x * CELL_SIZE + 8,
                y * CELL_SIZE + TOP_BAR + 8,
                CELL_SIZE - 16,
                CELL_SIZE - 16
            )

            pygame.draw.rect(screen, grid[y][x], rect, border_radius=14)

            # Hover highlight
            if (
                player_turn
                and not game_over
                and rect.collidepoint(mx, my)
                and can_capture(x, y, PLAYER)
            ):
                hover = pygame.Surface(rect.size, pygame.SRCALPHA)
                hover.fill((255, 255, 255, 40))
                screen.blit(hover, rect.topleft)

def draw_score():
    p = count_cells(PLAYER)
    a = count_cells(AI)
    text = font.render(f"You: {p}     AI: {a}", True, TEXT)
    screen.blit(text, (10, HEIGHT - BOTTOM_BAR + 10))

def draw_winner():
    overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
    overlay.fill((0, 0, 0, 180))
    screen.blit(overlay, (0, 0))

    msg = f"{winner} WINS!"
    text = big_font.render(msg, True, (255, 255, 255))
    screen.blit(
        text,
        (WIDTH//2 - text.get_width()//2, HEIGHT//2 - 40)
    )

# ================= AI =================
def ai_move():
    attacks = []
    moves = []

    for y in range(GRID_SIZE):
        for x in range(GRID_SIZE):
            if can_capture(x, y, AI):
                if grid[y][x] == PLAYER:
                    attacks.append((x, y))
                else:
                    moves.append((x, y))

    if attacks:
        x, y = random.choice(attacks)
    elif moves:
        x, y = random.choice(moves)
    else:
        return

    grid[y][x] = AI

# ================= MAIN LOOP =================
running = True
while running:
    clock.tick(FPS)
    screen.fill(BOARD_BG)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if (
            event.type == pygame.MOUSEBUTTONDOWN
            and player_turn
            and not game_over
        ):
            mx, my = event.pos
            gx = mx // CELL_SIZE
            gy = (my - TOP_BAR) // CELL_SIZE

            if 0 <= gx < GRID_SIZE and 0 <= gy < GRID_SIZE:
                if can_capture(gx, gy, PLAYER):
                    grid[gy][gx] = PLAYER
                    player_turn = False

    if not player_turn and not game_over:
        pygame.time.delay(350)
        ai_move()
        player_turn = True

    winner = check_win()
    if winner:
        game_over = True

    draw_turn_bar()
    draw_grid()
    draw_score()

    if game_over:
        draw_winner()

    pygame.display.flip()

pygame.quit()